In [1]:
from sklearn.neighbors import NearestNeighbors
from sklearn.model_selection import train_test_split as sklearn_train_test_split
from sklearn.preprocessing import LabelEncoder
from scipy.sparse import csr_matrix

import pandas as pd
import numpy as np
import zipfile

In [2]:
import warnings
warnings.filterwarnings('ignore')

### Загружаем наборы данных

In [3]:
ratings = pd.read_csv('./data/ratings.csv')
movies = pd.read_csv('./data/movies.csv')

In [4]:
ratings.head(5)

userId  movieId  rating   timestamp
0       1       31     2.5  1260759144
1       1     1029     3.0  1260759179
2       1     1061     3.0  1260759182
3       1     1129     2.0  1260759185
4       1     1172     4.0  1260759205

In [5]:
movies.head(5)

movieId                               title  \
0        1                    Toy Story (1995)   
1        2                      Jumanji (1995)   
2        3             Grumpier Old Men (1995)   
3        4            Waiting to Exhale (1995)   
4        5  Father of the Bride Part II (1995)   

                                        genres  
0  Adventure|Animation|Children|Comedy|Fantasy  
1                   Adventure|Children|Fantasy  
2                               Comedy|Romance  
3                         Comedy|Drama|Romance  
4                                       Comedy

### Создаём матрицу

In [6]:
def ratings_matrix(ratings):    
    # создайте csr матрицу 
    return csr_matrix(pd.crosstab(ratings.userId, ratings.movieId, ratings.rating, aggfunc=sum).fillna(0).values)  

R = ratings_matrix(ratings)

In [ ]:
# теперь это другая матрица)))

In [ ]:
ratings

In [ ]:
R

In [ ]:
R.indices

In [ ]:
R.data

# Пример моделей коллаборативной фильтрации (CF)

CF, так же известна, как "ближайшие соседи". Данный тип алгоритмов рекомендует на основе функции сходства между объектами или пользователями. Можно выделить основные варианты применения:

1. Поиск сходства между предметами или пользователями
2. Предикт рейтинга
3. Формирование Top-N


# 1. User-based (основано на пам

## Algorithm : user-to-user

Основаня идея алгоритма описана здесь<a href="https://romisatriawahono.net/lecture/rm/survey/information%20retrieval/Bobadilla%20-%20Recommender%20Systems%20-%202013.pdf">(J. Bobadilla et al. 2013)</a>.

### 1. Определим из всей выборки пользователе, которых будем сравнивать с активным

To find the $k$ most similar users to $u$, we use the cosine similarity and compute $w_{u,v}$ for all $v\in U$. Fortunately, libraries such as <i>scikit-learn (sklearn)</i> are very useful for such tasks :

1. Создадим модель ближайших соседей из sklearn в функции ```create_model()```. С кол-вом соседеней = 21, методом подбора "brute2.

2. Функция ```nearest_neighbors()``` возвращает knn пользователей для каждого пользователя.

In [7]:
def create_model(rating_matrix, metric):
    """
    - создание модели с базовыми параметрами
    """
    model = NearestNeighbors(metric=metric, n_neighbors=21, algorithm='brute')
    
    model.fit(rating_matrix)    
    return model

In [8]:
def nearest_neighbors(rating_matrix, model):
    """    
    :param rating_matrix : матрица рейтингов (nb_users, nb_items)
    :param model : модель knn  
    """    
    similarities, neighbors = model.kneighbors(rating_matrix)        
    return similarities[:, 1:], neighbors[:, 1:]

In [9]:
# метрику схожести успользуем Косинусную
model = create_model(rating_matrix=R, metric='cosine')
similarities, neighbors = nearest_neighbors(R, model)

### 2. Поиск элементов пользователя

In [10]:
def find_candidate_items(userid):
    """
    Поиск элементов для переданного пользователя
    
    :param userid : пользователь id
    :param neighbors : схожесть между пользователями      
    :return candidates : топ 10 элементов для пользователя
    """
    user_neighbors = neighbors[userid]
    
    activities = ratings.loc[ratings['userId'].isin(user_neighbors)]
    
    # сортируем элементы по частоте
    frequency = activities.groupby('movieId')['rating'].count().reset_index(name='count').sort_values(['count'],ascending=False)
    Gu_items = frequency['movieId']
    active_items = ratings.loc[ratings['userId'] == userid]['movieId'].to_list()
    candidates = np.setdiff1d(Gu_items, active_items, assume_unique=True)[:10]
        
    return candidates

### 3. Предикт рейтинга

Для предикта необходимо:

1. Сходство между пользвателями, которое получаем из функции ```nearest_neighbors()```
2. Нормализация рейтинга на общее среднее значение по пользователю $r_{v,i}-\bar{r}_v$.

In [11]:
# средний рейтинг по всем
mean = ratings.groupby(by='userId', as_index=False)['rating'].mean()
mean_ratings = pd.merge(ratings, mean, suffixes=('','_mean'), on='userId')

# нормализация рейтинга
mean_ratings['norm_rating'] = mean_ratings['rating'] - mean_ratings['rating_mean']

mean = mean.to_numpy()[:, 1]

In [12]:
np_ratings = mean_ratings.to_numpy()

```predict``` рейтинга между пользователями по функции:

\begin{equation}
 \hat{r}_{u,i}=\bar{r}_u + \frac{\sum_{v\in G_u}(r_{v,i}-\bar{r}_v)\cdot w_{u,v}}{\sum_{v\in G_u}|w_{u,v}|}.
\end{equation}

In [13]:
# берем пользователя 34
# берем фильм 780
# из всех кто смотрел фильм 780 мы берем только тех, кто является соседями для пользователя 34
# И казалось бы надо бы взять их средний рейтинг по этому фильму. (ну или средний рейтинг по фильму взвешенный по степени близости)
# Вместо этого мы берем их средние рейтинги (из расчета по всем фильмам) взвешиваем по степени близости и прибавляем к среднему рейтингу пользователя 34. 
# Кажется что здесь во-первых ошибка в индексах с самого начала (почти уверен, проверила на 34 узере), а во-вторых вот эта прибавка странная конечно
def predict(userid, itemid):
    """
    предикт для пользователя userid рейтинга на элемент itemid.
    
    :param
        - userid : пользователь для предикта
        - itemid : элемент для предикта
        
    :return
        - r_hat : предикт 
    """
    user_similarities = similarities[userid]
    user_neighbors = neighbors[userid]
    
    # средний рейтинг
    user_mean = mean[userid]
    
    # поиск пользователей, которые имеют рейтинг по элементу 'itemid'
    iratings = np_ratings[np_ratings[:, 1].astype('int') == itemid]
    
    # поиск похожих пользователей
    simus = iratings[np.isin(iratings[:, 0], user_neighbors)]
    
    # отбор похожих пользователей, которые имеют рейтинг по выбранному элементу
    normalized_ratings = simus[:,4]
    indexes = [np.where(user_neighbors == uid)[0][0] for uid in simus[:, 0].astype('int')]
    sims = user_similarities[indexes]
    
    num = np.dot(normalized_ratings, sims)
    den = np.sum(np.abs(sims))
    
    if num == 0 or den == 0:
        return user_mean
    
    # реализуем формулу предикта
    r_hat = user_mean + np.dot(normalized_ratings, sims) / np.sum(np.abs(sims))
    
    return r_hat

In [14]:
def user2userPredictions(userid, pred_path):
    """
    Сделаем предикт для каждого пользователя и сохраним в файл prediction.csv
    
    :param
        - userid : пользователя id
        - pred_path : куда сохраняем
    """    
    
    try:
        # поиск пользователей
        candidates = find_candidate_items(userid)

        # цикл по всем выбраным пользователям для предикта
        for itemid in candidates:

            # предикт для пользователя, по элементам
            r_hat = predict(userid, itemid)

            # сохраним
            with open(pred_path, 'a+') as file:
                line = '{},{},{}\n'.format(userid, itemid, r_hat)
                file.write(line)
    except IndexError:
        pass

In [15]:
import sys
import os

def user2userCF():
    """
    Предикт для всех пользователей, даже с 1 рейтингом   
    """
    # список всех пользователей
    users = ratings['userId'].unique()
    
    def _progress(count):
        sys.stdout.write('\rRating predictions. Progress status : %.1f%%' % (float(count/len(users))*100.0))
        sys.stdout.flush()
    
    saved_predictions = 'predictions.csv'    
    if os.path.exists(saved_predictions):
        os.remove(saved_predictions)
    
    for count, userid in enumerate(users):        
        # делаем предикт
        user2userPredictions(userid, saved_predictions)
        _progress(count)

In [16]:
user2userCF()

Rating predictions. Progress status : 99.9%

### 4. Top-N рекомендаций

Функция ```user2userRecommendation()``` делает отбор необходимых рекомендаций для пользователя

In [17]:
def user2userRecommendation(userid):
    """
    Делаем предикт для пользователя
    """
    
    saved_predictions = 'predictions.csv'
    
    predictions = pd.read_csv(saved_predictions, sep=',', names=['userId', 'movieId', 'predicted_rating'])
    predictions = predictions[predictions.userId==userid]
    List = predictions.sort_values(by=['predicted_rating'], ascending=False)
    
    List = pd.merge(List, movies, on='movieId', how='inner')
    
    return List

In [19]:
user2userRecommendation(34)

userId  movieId  predicted_rating  \
0      34      318          6.009687   
1      34        1          5.917152   
2      34      364          5.910717   
3      34      356          5.905854   
4      34     1210          5.896535   
5      34     1704          5.895702   
6      34      780          5.887666   
7      34     1073          5.842764   
8      34      648          5.829810   
9      34      736          5.796770   

                                               title  \
0                   Shawshank Redemption, The (1994)   
1                                   Toy Story (1995)   
2                              Lion King, The (1994)   
3                                Forrest Gump (1994)   
4  Star Wars: Episode VI - Return of the Jedi (1983)   
5                           Good Will Hunting (1997)   
6               Independence Day (a.k.a. ID4) (1996)   
7         Willy Wonka & the Chocolate Factory (1971)   
8                         Mission: Impossible (1996)   
9                                     Twister (1996)   

                                            genres  
0                                      Crime|Drama  
1      Adventure|Animation|Children|Comedy|Fantasy  
2  Adventure|Animation|Children|Drama|Musical|IMAX  
3                         Comedy|Drama|Romance|War  
4                          Action|Adventure|Sci-Fi  
5                                    Drama|Romance  
6                 Action|Adventure|Sci-Fi|Thriller  
7                  Children|Comedy|Fantasy|Musical  
8                Action|Adventure|Mystery|Thriller  
9                Action|Adventure|Romance|Thriller

## Хорошо бы это адаптировать просто под обычный User-based CF
#### Где рейтинг присваивается на основе просто средневзвешенного рейтинга ближайших соседей, которые посмотрели данный фильм.
